# Connecting to a Feature Store

特征存储是传统机器学习的一个概念，它确保输入模型的数据是最新的和相关的。有关更多信息，请参见此处。

在考虑将`LLM`应用程序投入生产时，这个概念非常重要。为了个性化`LLM`应用程序，您可能希望将`LLM`与有关特定用户的最新信息结合起来。特征存储是保持数据最新的好方法，`LangChain`提供了一种将该数据与`LLM`结合的简单方法。

在此笔记本中，我们将展示如何将提示模板连接到特征存储。基本思想是从提示模板内部调用特征存储来检索值，然后将这些值格式化为提示。


## Feast

To start, we will use the popular open source feature store framework [Feast](https://github.com/feast-dev/feast).

This assumes you have already run the steps in the README around getting started. We will build of off that example in getting started, and create and LLMChain to write a note to a specific driver regarding their up-to-date statistics.
首先，我们将使用流行的开源特征存储框架 Feast。
这假设您已经运行了`README`中关于入门的步骤。我们将在入门部分构建该示例，并创建和 LLMChain 以向特定驱动程序写入有关其最新统计数据的注释。


### Load Feast Store

Again, this should be set up according to the instructions in the Feast README


In [3]:
from feast import FeatureStore
import os

# You may need to update the path depending on where you stored it
feast_repo_path = os.path.abspath("feast/prompts_demo/feature_repo")
store = FeatureStore(repo_path=feast_repo_path)

### Prompts

Here we will set up a custom FeastPromptTemplate. This prompt template will take in a driver id, look up their stats, and format those stats into a prompt.

Note that the input to this prompt template is just `driver_id`, since that is the only user defined piece (all other variables are looked up inside the prompt template).


In [5]:
from langchain.prompts import PromptTemplate, StringPromptTemplate

In [10]:
template = """考虑到司机的最新统计数据，给他们写便条，将这些统计数据转发给他们。
如果他们的谈话率高于 0.5, 请称赞他们。 否则，在最后开个玩笑让他们感觉更好

以下是驱动程序统计数据：
Conversation rate: {conv_rate}
Acceptance rate: {acc_rate}
Average Daily Trips: {avg_daily_trips}

你的回应"""
prompt = PromptTemplate.from_template(template)

In [11]:
class FeastPromptTemplate(StringPromptTemplate):
    
    def format(self, **kwargs) -> str:
        driver_id = kwargs.pop("driver_id")
        feature_vector = store.get_online_features(
            features=[
                'driver_hourly_stats:conv_rate',
                'driver_hourly_stats:acc_rate',
                'driver_hourly_stats:avg_daily_trips'
            ],
            entity_rows=[{"driver_id": driver_id}]
        ).to_dict()
        kwargs["conv_rate"] = feature_vector["conv_rate"][0]
        kwargs["acc_rate"] = feature_vector["acc_rate"][0]
        kwargs["avg_daily_trips"] = feature_vector["avg_daily_trips"][0]
        return prompt.format(**kwargs)

In [12]:
prompt_template = FeastPromptTemplate(input_variables=["driver_id"])

In [13]:
print(prompt_template.format(driver_id=1001))

FeatureViewNotFoundException: Feature view driver_hourly_stats does not exist

### Use in a chain

We can now use this in a chain, successfully creating a chain that achieves personalization backed by a feature store


In [23]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

In [24]:
chain = LLMChain(llm=ChatOpenAI(), prompt=prompt_template)

In [25]:
chain.run(1001)

"Hi there! I wanted to update you on your current stats. Your acceptance rate is 0.055561766028404236 and your average daily trips are 936. While your conversation rate is currently 0.4745151400566101, I have no doubt that with a little extra effort, you'll be able to exceed that .5 mark! Keep up the great work! And remember, even chickens can't always cross the road, but they still give it their best shot."

## Tecton

Above, we showed how you could use Feast, a popular open source and self-managed feature store, with LangChain. Our examples below will show a similar integration using Tecton. Tecton is a fully managed feature platform built to orchestrate the complete ML feature lifecycle, from transformation to online serving, with enterprise-grade SLAs.


### Prerequisites

- Tecton Deployment (sign up at [https://tecton.ai](https://tecton.ai))
- `TECTON_API_KEY` environment variable set to a valid Service Account key


### Define and Load Features

We will use the user_transaction_counts Feature View from the [Tecton tutorial](https://docs.tecton.ai/docs/tutorials/tecton-fundamentals) as part of a Feature Service. For simplicity, we are only using a single Feature View; however, more sophisticated applications may require more feature views to retrieve the features needed for its prompt.

```python
user_transaction_metrics = FeatureService(
    name = "user_transaction_metrics",
    features = [user_transaction_counts]
)
```

The above Feature Service is expected to be [applied to a live workspace](https://docs.tecton.ai/docs/applying-feature-repository-changes-to-a-workspace). For this example, we will be using the "prod" workspace.


In [60]:
import tecton

workspace = tecton.get_workspace("prod")
feature_service = workspace.get_feature_service("user_transaction_metrics")

### Prompts

Here we will set up a custom TectonPromptTemplate. This prompt template will take in a user_id , look up their stats, and format those stats into a prompt.

Note that the input to this prompt template is just `user_id`, since that is the only user defined piece (all other variables are looked up inside the prompt template).


In [61]:
from langchain.prompts import PromptTemplate, StringPromptTemplate

In [77]:
template = """Given the vendor's up to date transaction stats, write them a note based on the following rules:

1. If they had a transaction in the last day, write a short congratulations message on their recent sales
2. If no transaction in the last day, but they had a transaction in the last 30 days, playfully encourage them to sell more.
3. Always add a silly joke about chickens at the end

Here are the vendor's stats:
Number of Transactions Last Day: {transaction_count_1d}
Number of Transactions Last 30 Days: {transaction_count_30d}

Your response:"""
prompt = PromptTemplate.from_template(template)

In [78]:
class TectonPromptTemplate(StringPromptTemplate):
    
    def format(self, **kwargs) -> str:
        user_id = kwargs.pop("user_id")
        feature_vector = feature_service.get_online_features(join_keys={"user_id": user_id}).to_dict()
        kwargs["transaction_count_1d"] = feature_vector["user_transaction_counts.transaction_count_1d_1d"]
        kwargs["transaction_count_30d"] = feature_vector["user_transaction_counts.transaction_count_30d_1d"]
        return prompt.format(**kwargs)

In [79]:
prompt_template = TectonPromptTemplate(input_variables=["user_id"])

In [80]:
print(prompt_template.format(user_id="user_469998441571"))

Given the vendor's up to date transaction stats, write them a note based on the following rules:

1. If they had a transaction in the last day, write a short congratulations message on their recent sales
2. If no transaction in the last day, but they had a transaction in the last 30 days, playfully encourage them to sell more.
3. Always add a silly joke about chickens at the end

Here are the vendor's stats:
Number of Transactions Last Day: 657
Number of Transactions Last 30 Days: 20326

Your response:


### Use in a chain

We can now use this in a chain, successfully creating a chain that achieves personalization backed by the Tecton Feature Platform


In [81]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

In [82]:
chain = LLMChain(llm=ChatOpenAI(), prompt=prompt_template)

In [83]:
chain.run("user_469998441571")

'Wow, congratulations on your recent sales! Your business is really soaring like a chicken on a hot air balloon! Keep up the great work!'

## Featureform

Finally, we will use [Featureform](https://github.com/featureform/featureform) an open-source and enterprise-grade feature store to run the same example. Featureform allows you to work with your infrastructure like Spark or locally to define your feature transformations.


### Initialize Featureform

You can follow in the instructions in the README to initialize your transformations and features in Featureform.


In [ ]:
import featureform as ff

client = ff.Client(host="demo.featureform.com")

### Prompts

Here we will set up a custom FeatureformPromptTemplate. This prompt template will take in the average amount a user pays per transactions.

Note that the input to this prompt template is just avg_transaction, since that is the only user defined piece (all other variables are looked up inside the prompt template).


In [ ]:
from langchain.prompts import PromptTemplate, StringPromptTemplate

In [ ]:
template = """Given the amount a user spends on average per transaction, let them know if they are a high roller. Otherwise, make a silly joke about chickens at the end to make them feel better

Here are the user's stats:
Average Amount per Transaction: ${avg_transcation}

Your response:"""
prompt = PromptTemplate.from_template(template)

In [ ]:
class FeatureformPromptTemplate(StringPromptTemplate):
    
    def format(self, **kwargs) -> str:
        user_id = kwargs.pop("user_id")
        fpf = client.features([("avg_transactions", "quickstart")], {"user": user_id})
        return prompt.format(**kwargs)

In [ ]:
prompt_template = FeatureformPrompTemplate(input_variables=["user_id"])

In [ ]:
print(prompt_template.format(user_id="C1410926"))

### Use in a chain

We can now use this in a chain, successfully creating a chain that achieves personalization backed by the Featureform Feature Platform


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

In [ ]:
chain = LLMChain(llm=ChatOpenAI(), prompt=prompt_template)

In [ ]:
chain.run("C1410926")